##### Copyright 2020 The TensorFlow IO Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Preparación y aumento de datos de audio

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/io/tutorials/audio"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/io/tutorials/audio.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/io/tutorials/audio.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver código fuente en GitHub</a>
</td>
      <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/io/tutorials/audio.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar el bloc de notas</a>
</td>
</table>

## Descripción general

Uno de los mayores desafíos del reconocimiento automático de voz es la preparación y el aumento de datos de audio. El análisis de datos de audio puede realizarse en el dominio del tiempo o de la frecuencia, lo que agrega más complejidad en comparación con otras fuentes de datos, como las imágenes.

Como parte del ecosistema TensorFlow, el paquete `tensorflow-io` proporciona bastantes API útiles relacionadas con el audio que ayudan a facilitar la preparación y el aumento de datos de audio.

## Preparación

### Instale los paquetes necesarios y reinicie el tiempo de ejecución

In [ ]:
!pip install tensorflow-io

## Uso

### Leer un archivo de audio

En TensorFlow IO, la clase `tfio.audio.AudioIOTensor` le permite leer un archivo de audio en un `IOTensor` con carga diferida:

In [ ]:
import tensorflow as tf
import tensorflow_io as tfio

audio = tfio.audio.AudioIOTensor('gs://cloud-samples-tests/speech/brooklyn.flac')

print(audio)

En el ejemplo anterior, el archivo Flac `brooklyn.flac` proviene de un clip de audio de acceso público en [Google Cloud](https://cloud.google.com/speech-to-text/docs/quickstart-gcloud).

La dirección de GCS `gs://cloud-samples-tests/speech/brooklyn.flac` se usa directamente porque GCS es un sistema de archivos compatible con TensorFlow. Además del formato `Flac`, `AudioIOTensor` también admite `WAV`, `Ogg`, `MP3` y `MP4A` con detección automática de formato de archivo.

`AudioIOTensor` tiene carga diferida, por lo que inicialmente solo se muestran la forma, el dtype y la frecuencia de muestreo. La forma del `AudioIOTensor` se representa como `[samples, channels]`, lo que significa que el clip de audio que usted cargó es un canal mono con `28979` muestras en `int16`.

El contenido del clip de audio solo se leerá según sea necesario, ya sea convirtiendo `AudioIOTensor` en `Tensor` mediante `to_tensor()` o mediante segmentación. La segmentación es especialmente útil cuando sólo se necesita una pequeña porción de un clip de audio grande:

In [ ]:
audio_slice = audio[100:]

# remove last dimension
audio_tensor = tf.squeeze(audio_slice, axis=[-1])

print(audio_tensor)

El audio se puede reproducir a través de:

In [ ]:
from IPython.display import Audio

Audio(audio_tensor.numpy(), rate=audio.rate.numpy())

Es más conveniente convertir el tensor en números flotantes y mostrar el clip de audio en un gráfico:

In [ ]:
import matplotlib.pyplot as plt


tensor = tf.cast(audio_tensor, tf.float32) / 32768.0

plt.figure()
plt.plot(tensor.numpy())

### Recorte el ruido

A veces tiene sentido recortar el ruido del audio, lo que se puede hacer mediante la API `tfio.audio.trim`. La API devuelve un par de posiciones `[start, stop]` del segmento:

In [ ]:
position = tfio.audio.trim(tensor, axis=0, epsilon=0.1)
print(position)

start = position[0]
stop = position[1]
print(start, stop)

processed = tensor[start:stop]

plt.figure()
plt.plot(processed.numpy())

### Fundido de entrada y de salida

Una técnica útil de ingeniería de audio es el fundido, que aumenta o disminuye gradualmente las señales de audio. Esto se puede hacer a través de `tfio.audio.fade`. `tfio.audio.fade` admite diferentes formas de fundido, como `linear`, `logarithmic` o `exponential`:

In [ ]:
fade = tfio.audio.fade(
    processed, fade_in=1000, fade_out=2000, mode="logarithmic")

plt.figure()
plt.plot(fade.numpy())

### Espectrograma

El procesamiento avanzado de audio suele funcionar con cambios de frecuencia en un lapso tiempo. En `tensorflow-io` una forma de onda se puede convertir en espectrograma a través de `tfio.audio.spectrogram`:

In [ ]:
# Convert to spectrogram
spectrogram = tfio.audio.spectrogram(
    fade, nfft=512, window=512, stride=256)

plt.figure()
plt.imshow(tf.math.log(spectrogram).numpy())

También es posible realizar transformaciones adicionales a diferentes escalas:

In [ ]:
# Convert to mel-spectrogram
mel_spectrogram = tfio.audio.melscale(
    spectrogram, rate=16000, mels=128, fmin=0, fmax=8000)


plt.figure()
plt.imshow(tf.math.log(mel_spectrogram).numpy())

# Convert to db scale mel-spectrogram
dbscale_mel_spectrogram = tfio.audio.dbscale(
    mel_spectrogram, top_db=80)

plt.figure()
plt.imshow(dbscale_mel_spectrogram.numpy())

### SpecAugment

Además de las API de aumento y preparación de datos mencionadas anteriormente, el paquete `tensorflow-io` también proporciona aumentos de espectrograma avanzados, en particular el enmascaramiento de frecuencia y tiempo que se analiza en [SpecAugment: un método simple de aumento de datos para el reconocimiento automático de voz (Park et al., 2019)](https://arxiv.org/pdf/1904.08779.pdf).

#### Enmascaramiento de frecuencia

En el enmascaramiento de frecuencia, los canales de frecuencia `[f0, f0 + f)` están enmascarados donde `f` se elige de una distribución uniforme de `0` para el parámetro de máscara de frecuencia `F`, y `f0` se elige de `(0, ν − f)` donde `ν` es la cantidad de canales de frecuencia.

In [ ]:
# Freq masking
freq_mask = tfio.audio.freq_mask(dbscale_mel_spectrogram, param=10)

plt.figure()
plt.imshow(freq_mask.numpy())

#### Enmascaramiento de tiempo

En el enmascaramiento de tiempo, se enmascaran `t` pasos de tiempo consecutivos `[t0, t0 + t)` donde `t` se elige de una distribución uniforme de `0` para el parámetro de máscara de tiempo `T`, y `t0` se elige de `[0, τ − t)` donde `τ` es los pasos de tiempo.

In [ ]:
# Time masking
time_mask = tfio.audio.time_mask(dbscale_mel_spectrogram, param=10)

plt.figure()
plt.imshow(time_mask.numpy())